In [1]:
import os
from tensorflow.keras.models import load_model

new_model = load_model(r'C:\Users\HP\FER_SL\models\affectnet_emotions\mobilenet_7.h5')
new_model.summary()

def mobilenet_preprocess_input(x,**kwargs):
        x[..., 0] -= 103.939
        x[..., 1] -= 116.779
        x[..., 2] -= 123.68
        return x
preprocessing_function=mobilenet_preprocess_input

INPUT_SIZE = (224,224)
USE_PREPROCESSING=True
print(INPUT_SIZE)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [2]:
import tensorflow as tf
import pandas as pd
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
from tqdm import tqdm
import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model,Sequential, load_model,model_from_json
from tensorflow.keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,vgg16,resnet_v2,resnet
#from tensorflow.keras.utils.generic_utils import CustomObjectScope
from tensorflow.keras.layers import Flatten, Dense, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
from tensorflow.keras.metrics import top_k_categorical_accuracy
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.compat.v1.keras.backend import set_session 


In [3]:
new_model.load_weights(r'C:\Users\HP\FER_SL\models\affectnet_emotions\mobilenet_7.h5')

In [4]:
target_size = (224, 224)
BATCH_SIZE = 80

test_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

test_set = test_datagen.flow_from_directory(r'C:\Users\HP\FER_SL\sl_data\test_files_cropped', target_size=target_size,
                                            batch_size=BATCH_SIZE, class_mode='categorical',follow_links=True)

Found 507 images belonging to 7 classes.


In [5]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [6]:
idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}

In [7]:
from facial_analysis import FacialImageProcessing
imgProcessing=FacialImageProcessing(False)

In [8]:
import numpy as np
y_val=np.array(test_set.labels)
y_scores_val=[]
#new_model.load_weights(net_description+'_ft.h5')
for filepath in log_progress(test_set.filepaths):
    img=cv2.imread(filepath)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,INPUT_SIZE)
    if USE_PREPROCESSING:
        inp = preprocessing_function(np.expand_dims(img, axis=0).astype(np.float32))
    else:
        inp=img.astype(np.float32)
        inp[..., 0] -= 103.939
        inp[..., 1] -= 116.779
        inp[..., 2] -= 123.68
        inp = np.expand_dims(inp, axis=0)

    scores=new_model.predict(inp)
    #print(scores)
    y_scores_val.append(scores)

y_scores_val=np.concatenate(y_scores_val, axis=0)
#print(y_scores_val.shape,y_val.shape)
     
        
    

In [9]:
y_pred=np.argmax(y_scores_val,axis=1)
acc=100.0*(y_val==y_pred).sum()/len(y_val)
print(acc)

targets = ['anger','disgust','fear','happy','neutral','sad','surprise']

for i in range(y_scores_val.shape[1]):
    _val_acc=(y_pred[y_val==i]==i).sum()/(y_val==i).sum()
    print('%s acc: %f' %(idx_to_class[i],100*_val_acc))

52.071005917159766
Anger acc: 54.545455
Disgust acc: 74.193548
Fear acc: 26.315789
Happiness acc: 15.789474
Neutral acc: 26.190476
Sadness acc: 11.864407
Surprise acc: 79.640719


In [10]:
from sklearn.metrics import classification_report

target_names = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', 'Surprise']
print(classification_report(y_val, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Anger       0.82      0.55      0.65       132
     Disgust       0.21      0.74      0.33        31
        Fear       0.32      0.26      0.29        57
       Happy       0.17      0.16      0.16        19
     Neutral       0.58      0.26      0.36        42
     Sadness       0.29      0.12      0.17        59
    Surprise       0.66      0.80      0.72       167

    accuracy                           0.52       507
   macro avg       0.44      0.41      0.38       507
weighted avg       0.57      0.52      0.52       507



In [11]:
# Weighted accuracy

from sklearn.metrics import f1_score
w_acc = f1_score(y_val, y_pred, average='weighted')
print(w_acc)

0.516465815946657


# Confusion Matrix

In [12]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_val, y_pred, normalize='true')
print(idx_to_class.values())
print(cm)

dict_values(['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise'])
[[0.54545455 0.27272727 0.07575758 0.         0.01515152 0.00757576
  0.08333333]
 [0.06451613 0.74193548 0.03225806 0.09677419 0.         0.
  0.06451613]
 [0.         0.31578947 0.26315789 0.         0.01754386 0.05263158
  0.35087719]
 [0.         0.05263158 0.         0.15789474 0.         0.05263158
  0.73684211]
 [0.02380952 0.07142857 0.04761905 0.23809524 0.26190476 0.0952381
  0.26190476]
 [0.18644068 0.33898305 0.08474576 0.03389831 0.06779661 0.11864407
  0.16949153]
 [0.01197605 0.05389222 0.08383234 0.         0.00598802 0.04790419
  0.79640719]]


# Sensitivity

In [13]:
sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])
print('Sensitivity : ', sensitivity )

#TP/(TP+FN) - Sensitivity per class

Sensitivity :  0.6607142857142857


In [14]:
# Sensitivity per class

from sklearn.metrics import recall_score
sensitivity_per_class = recall_score(y_val,y_pred,average=None)
print(sensitivity_per_class)

[0.54014599 0.74193548 0.26315789 0.15789474 0.26190476 0.11864407
 0.79640719]
